# Monthly Bill Generator
This notebook implements a function to generate a monthly billing summary from a list of billing items. It includes date overlap checks, proration of billing amounts, and grouping of similar billing entries.

In [ ]:
from datetime import datetime
from collections import defaultdict
import calendar

In [ ]:

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    year, month = map(int, target_month.split("-"))
    month_start = datetime(year, month, 1).date()
    month_end = datetime(year, month, calendar.monthrange(year, month)[1]).date()

    groups = defaultdict(lambda: {'qty': 0, 'amount': 0.0})

    for item in item_list:
        start = datetime.strptime(item['start_date'], "%Y-%m-%d").date()
        stop = datetime.strptime(item['stop_date'], "%Y-%m-%d").date()
        rate = float(item['rate'])
        qty = int(item['qty'])

        # Determine overlap
        active_start = max(start, month_start)
        active_end = min(stop, month_end)

        if active_start > active_end:
            continue  # Not active in this month

        active_days = (active_end - active_start).days + 1
        total_days = (month_end - month_start).days + 1
        period = f"{active_start} to {active_end}"

        prorated_amount = round((active_days / total_days) * rate * qty, 2)

        group_key = (item['item_code'], rate, period)
        groups[group_key]['qty'] += qty
        groups[group_key]['amount'] += prorated_amount

    line_items = []
    total_revenue = 0.0
    for (item_code, rate, billing_period), values in groups.items():
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": values['qty'],
            "amount": round(values['amount'], 2),
            "billing_period": billing_period
        })
        total_revenue += values['amount']

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }


In [ ]:

# Example usage
items = [
    {"item_code": "A101", "start_date": "2024-01-15", "stop_date": "2024-03-10", "rate": 100.0, "qty": 2},
    {"item_code": "A101", "start_date": "2024-03-01", "stop_date": "2024-03-31", "rate": 100.0, "qty": 1},
    {"item_code": "B205", "start_date": "2024-03-15", "stop_date": "2024-03-25", "rate": 200.0, "qty": 1}
]

result = generate_monthly_bill(items, "2024-03")
print("Line Items:")
for line in result['line_items']:
    print(line)
print("Total Revenue:", result['total_revenue'])
